In [24]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from surprise import Dataset,accuracy,Reader,SVD,SVDpp,KNNBasic,KNNWithZScore,KNNWithMeans
from surprise.model_selection import cross_validate,train_test_split,GridSearchCV,KFold
from surprise.accuracy import rmse
from collections import defaultdict
from sklearn.metrics import r2_score, accuracy_score, mean_squared_error

pd.set_option('display.width', 100)
pd.set_option('display.max_colwidth', 1)
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
model_df=pd.read_csv("processed_data.csv")
reader = Reader(rating_scale=(1,5))

In [3]:
data = Dataset.load_from_df(model_df[['student_id', 'course_id', 'course_rating']], reader)
trainSet,testSet=train_test_split(data,test_size=.2)

In [21]:
svd_params = {'n_factors': [100,150],
              'n_epochs': [5,10,15],
              'lr_all':[0.005,0.01,0.1],
              'reg_all':[0.02,0.05,0.1]}



grid_search_svd = GridSearchCV(SVDpp,svd_params,measures=['rmse','mae'], cv=5)

grid_search_svd.fit(data)
print(grid_search_svd.best_params['rmse'])
print(grid_search_svd.best_score['rmse'])
print(grid_search_svd.best_score['mae'])

algo_grid_search_svd = grid_search_svd.best_estimator['rmse']
cross_validate(algo_grid_search_svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

predictions = algo_grid_search_svd.test(testSet)
pred = pd.DataFrame(predictions)
pred.sort_values(by=['est'],ascending = False).head(5)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix.

,uid,iid,r_ui,est,details
884,693,54,2.0,5.000000,"{'actual_k': 1, 'was_impossible': False}"
26,3852,17,4.0,5.000000,"{'actual_k': 1, 'was_impossible': False}"
638,3454,1,4.0,4.899731,"{'actual_k': 40, 'was_impossible': False}"
890,2382,42,3.0,4.666667,"{'actual_k': 3, 'was_impossible': False}"
308,1309,11,5.0,4.500000,"{'actual_k': 2, 'was_impossible': False}"


In [15]:
print("Validation RMSE:",accuracy.rmse(predictions))

RMSE: 0.8440
Validation RMSE: 0.8439802793011574


In [30]:
def precision_recall_at_k(predictions, k=5, threshold=3.5):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls

sim_options = {"name": "pearson_baseline", "shrinkage": 0}

kf = KFold(n_splits=5)
algo = NMF(n_factors=100,n_epochs=50)
#algo = KNNWithZScore(min_k=3,k=10,user_based=True,sim_options=sim_options)
prec_to_ave = []
rec_to_ave = []

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=3.5)

    # Precision and recall can then be averaged over all users
    
    prec_to_ave.append(sum(prec for prec in precisions.values()) / len(precisions))
    rec_to_ave.append(sum(rec for rec in recalls.values()) / len(recalls))

In [31]:
def make_binary_tpr_fpr(predictions, threshold=3.5):
    pred_df = pd.DataFrame(predictions)
    pred_df['r_ui'].where(pred_df['r_ui']>threshold, 1, inplace=True)
    pred_df['r_ui'].where(pred_df['r_ui']<=threshold, 0, inplace=True)
    
    pred_df['est'].where(pred_df['est']>threshold, 1, inplace=True)
    pred_df['est'].where(pred_df['est']<=threshold, 0, inplace=True)

    return pred_df['r_ui'], pred_df['est'], 

true_r, est = make_binary_tpr_fpr(predictions)

precision_average = sum(prec_to_ave)/len(prec_to_ave)
recall_average = sum(rec_to_ave)/len(prec_to_ave)

print("Precision and Recall averages are {0} and {1}, respectively".format(precision_average, recall_average))

Precision and Recall averages are 0.6766921853408656 and 0.8720417238713847, respectively


In [ ]:
#SVD Precision and Recall averages are 0.7046537476761795 and 0.8274135773926707, respectively
#SVDpp Precision and Recall averages are 0.7052122959393353 and 0.8262506348046192, respectively
#KNNBasic Precision and Recall averages are 0.6396854895155115 and 0.9335498455647226, respectively
#mEAN Precision and Recall averages are 0.7048398766024768 and 0.8201545929357783, respectively
#z Precision and Recall averages are 0.6996333345327741 and 0.816583752303025, respectively
# Precision and Recall averages are 0.6766921853408656 and 0.8720417238713847, respectively